In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras as k
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.losses import mean_absolute_error,mean_squared_error
from keras import backend as K
import tensorflow as tf

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
K.tensorflow_backend._get_available_gpus()

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [2]:
df = pd.read_csv("../input/train_V2.csv")
df_test = pd.read_csv("../input/test_V2.csv")
out = pd.read_csv("../input/sample_submission_V2.csv")

In [3]:
df.head()

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
0,7f96b2f878858a,4d4b580de459be,a10357fd1a4a91,0,0,0.00,0,0,0,60,1241,0,0,0.00,1306,squad-fpp,28,26,-1,0,0.0000,0,0.00,0,0,244.80,1,1466,0.4444
1,eef90569b9d03c,684d5656442f9e,aeb375fc57110c,0,0,91.47,0,0,0,57,0,0,0,0.00,1777,squad-fpp,26,25,1484,0,0.0045,0,11.04,0,0,1434.00,5,0,0.6400
2,1eaf90ac73de72,6a4a42c3245a74,110163d8bb94ae,1,0,68.00,0,0,0,47,0,0,0,0.00,1318,duo,50,47,1491,0,0.0000,0,0.00,0,0,161.80,2,0,0.7755
3,4616d365dd2853,a930a9c79cd721,f1f1f4ef412d7e,0,0,32.90,0,0,0,75,0,0,0,0.00,1436,squad-fpp,31,30,1408,0,0.0000,0,0.00,0,0,202.70,3,0,0.1667
4,315c96c26c9aac,de04010b3458dd,6dc8ff871e21e6,0,0,100.00,0,0,0,45,0,1,1,58.53,1424,solo-fpp,97,95,1560,0,0.0000,0,0.00,0,0,49.75,2,0,0.1875


In [4]:
df[df['winPlacePerc'].isnull()]
df.drop(2744604, inplace=True)
df[df['winPlacePerc'].isnull()]

,Id,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,kills,killStreaks,longestKill,matchDuration,matchType,maxPlace,numGroups,rankPoints,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc


In [5]:
df['movement'] = df['swimDistance'] + df['walkDistance'] + df['rideDistance']
df.drop(['swimDistance','walkDistance','rideDistance'], inplace=True, axis=1)
column_names = ['assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'maxPlace',
       'numGroups', 'revives', 'roadKills', 'teamKills',
       'vehicleDestroys', 'weaponsAcquired', 'winPoints','movement']
x = df[column_names]
y = df['winPlacePerc']
x.shape

(4446965, 20)

In [ ]:
for i in x:
    de = np.max(x[i]) - np.min(x[i])
    x[i] -= np.min(x[i])
    x[i] = x[i] / (de)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 705023)

In [ ]:
m = Sequential()
m.add(Dense(256, input_dim = 20, init = 'uniform'))
m.add(BatchNormalization())
m.add(Activation('relu'))
m.add(Dropout(0.5))

m.add(Dense(256, init = 'uniform'))
m.add(BatchNormalization())
m.add(Activation('relu'))
m.add(Dropout(0.5))

m.add(Dense(256, init = 'uniform'))
m.add(BatchNormalization())
m.add(Activation('relu'))
m.add(Dropout(0.5))

m.add(Dense(256, init = 'uniform'))
m.add(BatchNormalization())
m.add(Activation('relu'))
m.add(Dropout(0.5))

m.add(Dense(256, init = 'uniform'))
m.add(BatchNormalization())
m.add(Activation('relu'))

m.add(Dense(1, init = 'uniform'))
m.add(BatchNormalization())
m.add(Activation('sigmoid'))
m.summary()

## Baseline model
### Neural Network

In [ ]:
m.compile(loss='mae', optimizer=Adam(lr=0.05 ), metrics=['mae'])
es = EarlyStopping(monitor='val_loss',patience=8,baseline=None, restore_best_weights=False)

In [ ]:
hist = m.fit(x_train,y_train,batch_size=500, validation_split=0.1,epochs=70)

In [ ]:
df_test['movement'] = df_test['swimDistance'] + df_test['walkDistance'] + df_test['rideDistance']
df_test.drop(['swimDistance','walkDistance','rideDistance'], inplace=True, axis=1)
x = df_test[column_names]

for i in x:
    de = np.max(x[i]) - np.min(x[i])
    x[i] -= np.min(x[i])
    x[i] = x[i] / (de)


pred = m.predict(x)
out['winPlacePerc'] = pred
out.to_csv("myans.csv",index=False)